In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import files
uploaded = files.upload()

df1 = pd.read_csv('Train.csv')

#country = pd.read_excel('Country-Code.xlsx')
#country.head()
df1.shape
df1.isnull().sum()


Saving Train.csv to Train.csv


id                                   0
name                                12
host_id                              0
host_name                           13
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       6982
reviews_per_month                 6982
calculated_host_listings_count       0
availability_365                     0
dtype: int64

# Modelling

# 1. DATA CLEANING : Removing unneessary columns (except 'host_id', neighbourhood_group',latitude', 'longitude', 'room_type', 'price')

In [ ]:
df1.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [ ]:
X_enc_rt = pd.get_dummies(df1.room_type, prefix='Roomtype')
X_enc_rt.head()
X_enc_nhg = pd.get_dummies(df1.neighbourhood_group, prefix='NBG')

df11 = pd.concat([df1, X_enc_rt.drop('Roomtype_Shared room',axis='columns')], axis=1)
df11 = pd.concat([df11, X_enc_nhg.drop('NBG_Staten Island',axis='columns')], axis=1)

df11.shape



(34226, 22)

In [ ]:
#df2=df1.drop(['id', 'name', 'host_name', 'neighbourhood_group',
#       'neighbourhood','minimum_nights', 'number_of_reviews', 'last_review',
#       'reviews_per_month', 'calculated_host_listings_count',
#       'availability_365'], axis='columns')
# did not work we can only drop 8 columns at a time

In [ ]:
df2=df11.drop(['id', 'name', 'host_name',
       'neighbourhood','minimum_nights', 'number_of_reviews'], axis='columns')
df2.shape

(34226, 16)

In [ ]:
df3=df2.drop(['last_review','reviews_per_month', 'calculated_host_listings_count','availability_365'], axis='columns')

In [ ]:
df3.shape
#df2.columns

(34226, 12)

In [ ]:
df3.isnull().sum() # no null values in df3

host_id                     0
neighbourhood_group         0
latitude                    0
longitude                   0
room_type                   0
price                       0
Roomtype_Entire home/apt    0
Roomtype_Private room       0
NBG_Bronx                   0
NBG_Brooklyn                0
NBG_Manhattan               0
NBG_Queens                  0
dtype: int64

# dropping the column "room_type"

In [ ]:
df4 = df3.drop(['room_type'],axis='columns')
df4.head()

,host_id,neighbourhood_group,latitude,longitude,price,Roomtype_Entire home/apt,Roomtype_Private room,NBG_Bronx,NBG_Brooklyn,NBG_Manhattan,NBG_Queens
0,4271676,Manhattan,40.72217,-73.99481,120,0,1,0,0,1,0
1,11460768,Manhattan,40.80020,-73.96045,1500,1,0,0,0,1,0
2,267193767,Queens,40.75558,-73.89316,200,1,0,0,0,0,1
3,43392243,Staten Island,40.60110,-74.07830,30,0,0,0,0,0,0
4,16065171,Manhattan,40.81022,-73.94266,115,0,0,0,0,1,0


# 2 REMOVING OUTLIERS

In [ ]:
df4.describe()


,host_id,latitude,longitude,Roomtype_Entire home/apt,Roomtype_Private room,NBG_Bronx,NBG_Brooklyn,NBG_Manhattan,NBG_Queens
count,1.466900e+04,14669.000000,14669.000000,14669.000000,14669.000000,14669.000000,14669.000000,14669.000000,14669.000000
mean,6.827691e+07,40.728693,-73.952402,0.516736,0.458927,0.022087,0.413866,0.440930,0.114595
std,7.925935e+07,0.054359,0.046148,0.499737,0.498327,0.146973,0.492542,0.496515,0.318544
min,2.787000e+03,40.506410,-74.242850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.809661e+06,40.690160,-73.982890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.121494e+07,40.723220,-73.955640,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.074344e+08,40.762950,-73.936860,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000
max,2.743213e+08,40.908040,-73.716900,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Outlier Removal (Retaining 5% -95% quantile prices) 

In [ ]:

max = df4['price'].quantile(0.95)
min = df4['price'].quantile(0.05)

df6 = df4[(df4['price']<max) &(df4['price']>min) ]
#df6 = remove_price_outliers(df5)
df6.head()
print('max,min', min,max)

max,min 40.0 360.0


In [ ]:
df7 = df6.drop(['neighbourhood_group'],axis='columns')
df7outl = df4.drop(['neighbourhood_group'],axis='columns')
df7.shape

(30560, 10)

# 3 LINEAR REGRESSION (RMSE calculation)

In [ ]:

X=df7.drop('price', axis='columns')
X.head()
X.shape
X_outliers = df7outl.drop('price', axis='columns')

In [ ]:
y = df7.price
y = np.log10(y)
y.shape
y.head()
y_outliers = df7outl.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=1000)

#y_outliers = df4.drop(['neighbourhood_group'],axis='columns').price
#X_outliers = df4.drop(['neighbourhood_group'],axis='columns') 
X1_train,X_outliers_test,y_outliers_train,y_outliers_test=train_test_split(X_outliers,y_outliers,test_size=0.4, random_state=1000)

y_outliers_test.min()


# X_outliers_test.head()

0

# Transforming test data for validation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
sklr = LinearRegression(normalize=False)
lm = sklr.fit(X_train,y_train)


score = sklr.score(X_test,y_test)


preds_test = sklr.predict(X_test)
preds_train = sklr.predict(X_train)



rmse_test = np.sqrt(mean_squared_error(10**y_test, 10**preds_test))
rmse_train = np.sqrt(mean_squared_error(10**y_train, 10**preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)


preds_test2 = sklr.predict(X_outliers_test)



rmse_test2 = np.sqrt(mean_squared_error(y_outliers_test, 10**preds_test2))


print("RMSE_test_outliers: %f" % (rmse_test2))




RMSE_test: 55.944859
RMSE_train: 54.477939
Score: 0.4981345377875511
RMSE_test_outliers: 215.180296


Results without WITH LOG

RMSE: 1.393696

Train Value : [ 39.00 - 100.00 -  93.00 -  80.00  - 98.00]

Pretiction  : [ 58.29 - 107.01 - 142.65 - 88.08 - 88.26]

Test Value : [ 55.00 -  45.00 -  60.00 - 150.00 - 195.00]

Pretiction [ 72.38 - 60.98 - 106.95- 86.79 - 144.76]

Results without LOG

RMSE: 36.597191

Train Value : [ 39.00 100.00  93.00  80.00  98.00]

Pretiction  : [ 61.31 121.75 147.80  96.31  98.87 ]

Test Value : [ 55.00 - 45.00 - 60.00 - 150.00 - 195.00]

Pretiction [ 76.60 -   64.40 - 127.06 - 97.01 - 151.19]



 # 4 XG Boost regression (RMSE Calculation)

In [ ]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

lm=  xg_reg.fit(X_train,y_train)

score = xg_reg.score(X_test,y_test)


preds_test = xg_reg.predict(X_test)
preds_train = xg_reg.predict(X_train)



rmse_test = np.sqrt(mean_squared_error(10**y_test, 10**preds_test))
rmse_train = np.sqrt(mean_squared_error(10**y_train, 10**preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)


preds_test2 = xg_reg.predict(X_outliers_test)



rmse_test2 = np.sqrt(mean_squared_error(y_outliers_test, 10**preds_test2))

print("RMSE_test_outliers: %f" % (rmse_test2))


[07:43:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE_test: 117.561943
RMSE_train: 117.067078
Score: -4.913664853806872
RMSE_test_outliers: 250.912650


# Lasso 

In [ ]:
from sklearn.linear_model import Lasso
skls = Lasso()
lm = skls.fit(X_train,y_train)


score = skls.score(X_test,y_test)


preds_test = skls.predict(X_test)
preds_train = skls.predict(X_train)



rmse_test = np.sqrt(mean_squared_error(10**y_test, 10**preds_test))
rmse_train = np.sqrt(mean_squared_error(10**y_train, 10**preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)


preds_test2 = skls.predict(X_outliers_test)



rmse_test2 = np.sqrt(mean_squared_error(y_outliers_test, 10**preds_test2))


print("RMSE_test_outliers: %f" % (rmse_test2))

RMSE_test: 73.765281
RMSE_train: 72.532547
Score: -0.0005523784502321494
RMSE_test_outliers: 226.204371


 # DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
skdt = DecisionTreeRegressor()
lm = skdt.fit(X_train,y_train)

score = skdt.score(X_test,y_test)


preds_test = skdt.predict(X_test)
preds_train = skdt.predict(X_train)



rmse_test = np.sqrt(mean_squared_error(10**y_test, 10**preds_test))
rmse_train = np.sqrt(mean_squared_error(10**y_train, 10**preds_train))

print("RMSE_test: %f" % (rmse_test))
print("RMSE_train: %f" % (rmse_train))

print ('Score:', score)


preds_test2 = skdt.predict(X_outliers_test)



rmse_test2 = np.sqrt(mean_squared_error(y_outliers_test, 10**preds_test2))


print("RMSE_test_outliers: %f" % (rmse_test2))

RMSE_test: 70.312169
RMSE_train: 0.000000
Score: 0.19783262121514664
RMSE_test_outliers: 208.608683


# Random forest with searchCV


In [ ]:
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor ,RandomForestRegressor

rndf_model = RandomForestRegressor(random_state=33)
par_grid = dict(n_estimators=range(15,100,15),max_depth=range(4,25),min_samples_split=range(60,500,20),min_samples_leaf=range(25,75,15),max_leaf_nodes=range(7,30,2))

scanCV = RandomizedSearchCV(rndf_model,par_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,random_state=33)
lm = scanCV.fit(X_train,y_train)

preds_test =scanCV.best_estimator_.predict(X_test)

print('RMSE_test:',np.sqrt(mean_squared_error(10**y_test, 10**preds_test)))

preds_train = scanCV.best_estimator_.predict(X_train)
print('RMSE_train:',np.sqrt(mean_squared_error(10**y_train, 10**preds_train)))

preds_test2 = scanCV.best_estimator_.predict(X_outliers_test)
print('RMSE_test_outliers:',np.sqrt(mean_squared_error(y_outliers_test, 10**preds_test2)))


RMSE_test: 53.38246382943514
RMSE_train: 51.67327674747896
RMSE_test_outliers: 213.91269770746572


# Finding the best model using 'sklearn' GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import ShuffleSplit



In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' :{
            'model' : LinearRegression(),
            'params': {
                'normalize' : [True, False]
            }
        },
        'lasso' : {
            'model' : Lasso(),
            'params': {
                'alpha' : [1,2],
                'selection' : ['random','cyclic']
            }
        },
        'decision_tree' : {
            'model' : DecisionTreeRegressor(),
            'params' : {
                'criterion' : ['mse', 'friedman_mse'],
                'splitter'  : ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0 )
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model' : algo_name,
            'best_score' : gs.best_score_,
            'best_params' : gs.best_params_,
        })
    return pd.DataFrame(scores,columns = ['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,y)

,model,best_score,best_params
0,linear_regression,0.093290,{'normalize': True}
1,lasso,0.086391,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,-0.818443,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


# Predictions on TEST data

In [ ]:
from google.colab import files
uploaded = files.upload()

df_t = pd.read_csv('Test.csv')


Saving Test.csv to Test.csv


In [ ]:
X_enc_rt = pd.get_dummies(df_t.room_type, prefix='Roomtype')
X_enc_rt.head()
X_enc_nhg = pd.get_dummies(df_t.neighbourhood_group, prefix='NBG')

df12 = pd.concat([df_t, X_enc_rt.drop('Roomtype_Shared room',axis='columns')], axis=1)
df12 = pd.concat([df12, X_enc_nhg.drop('NBG_Staten Island',axis='columns')], axis=1)

df12.shape



(14669, 21)

In [ ]:
df2=df12.drop(['id', 'name', 'host_name',
       'neighbourhood','minimum_nights', 'number_of_reviews'], axis='columns')
df3=df2.drop(['last_review','reviews_per_month', 'calculated_host_listings_count','availability_365'], axis='columns')
df4 = df3.drop(['room_type'],axis='columns')
df4.head()
X = df4.drop(['neighbourhood_group'],axis='columns')
X.shape
Test_data_predictions = scanCV.best_estimator_.predict(X)

#final_submission['id'] = df12['id']

#final_submission['price'] = pd.DataFrame(10**Test_data_predictions)



submission = pd.DataFrame(10**Test_data_predictions,index = df12['id'],columns = ['price'])

#print(final_submission2.columns)

submission.to_csv('Submission_Bombay_Hackers.csv')


submission.head()

,price
id,
1525602,128.354989
30430185,136.323322
21354525,87.066393
35995074,65.665325
34392081,82.985592
